# 5.0 – Simulación de Data Drift y Evaluación de Performance
En esta notebook simulamos un cambio de distribución (*data drift*), 
aplicamos pruebas estadísticas (KS y Chi-cuadrada) y medimos la 
degradación del modelo respecto a la línea base.

Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import ks_2samp, chi2_contingency

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    roc_auc_score,
    RocCurveDisplay,
    PrecisionRecallDisplay
)

import joblib
plt.rcParams["figure.figsize"] = (6,4)
plt.rcParams["axes.grid"] = True

Cargar modelo y datos

In [ ]:
# Cargar modelo / pipeline entrenado
# AJUSTA la ruta y el nombre del archivo del modelo
model = joblib.load("models/best_model.pkl")  # <-- AJUSTAR si es necesario

# Cargar dataset de validación / test
# Usa el mismo que emplearon en 3.0 para evaluar el modelo
df = pd.read_csv("data/processed/test.csv")   # <-- AJUSTAR ruta

target_col = "NObeyesdad"                     # <-- AJUSTAR si tu target tiene otro nombre

X = df.drop(columns=[target_col])
y = df[target_col]

num_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

print("Variables numéricas:", num_cols)
print("Variables categóricas:", cat_cols)

Métricas base del modelo (sin drift)

In [ ]:
y_pred = model.predict(X)

baseline_metrics = {
    "accuracy": accuracy_score(y, y_pred),
    "f1_macro": f1_score(y, y_pred, average="macro")
}

# Si el modelo soporta predict_proba, calculamos ROC AUC
if hasattr(model, "predict_proba"):
    y_prob = model.predict_proba(X)
    # Para multiclase usamos ovo
    baseline_metrics["roc_auc_ovo"] = roc_auc_score(
        y, y_prob, multi_class="ovo"
    )

baseline_metrics

Generar dataset con DRIFT

In [ ]:
df_drift = df.copy()

# --- Ejemplos de drift numérico (ajusta nombres de columnas) ---
for col in ["Age", "Weight", "Height"]:  # <-- AJUSTAR a tus columnas reales
    if col in df_drift.columns:
        # Movemos la media hacia arriba y agregamos ruido
        df_drift[col] = df_drift[col] + np.random.normal(loc=5, scale=3, size=len(df_drift))

# --- Ejemplo de drift categórico ---
# Cambiamos la proporción de fumadores/no fumadores
if "SMOKE" in df_drift.columns:          # <-- AJUSTAR a tu columna categórica
    df_drift.loc[df_drift["SMOKE"] == "yes", "SMOKE"] = "no"

X_drift = df_drift.drop(columns=[target_col])
y_drift = df_drift[target_col]

df_drift.head()

KS test para variables numéricas

results_num = []

for col in num_cols:
    stat, p = ks_2samp(X[col], X_drift[col])
    results_num.append([col, stat, p])

ks_df = pd.DataFrame(results_num, columns=["variable", "ks_statistic", "p_value"])
ks_df.sort_values("p_value").head(10)

Variables numéricas con drift (criterios: p < 0.05 y KS > 0.1):

In [ ]:
drift_num = ks_df[(ks_df["p_value"] < 0.05) & (ks_df["ks_statistic"] > 0.1)]
drift_num

Chi-cuadrada para categóricas

In [ ]:
results_cat = []

for col in cat_cols:
    contingency = pd.crosstab(X[col], X_drift[col])
    stat, p, dof, exp = chi2_contingency(contingency)
    results_cat.append([col, stat, p])

chi_df = pd.DataFrame(results_cat, columns=["variable", "chi2_statistic", "p_value"])
chi_df.sort_values("p_value").head(10)

Variables categóricas con drift (p < 0.05):

In [ ]:
drift_cat = chi_df[chi_df["p_value"] < 0.05]
drift_cat

Métricas del modelo con DRIFT

In [ ]:
y_pred_drift = model.predict(X_drift)

drift_metrics = {
    "accuracy": accuracy_score(y_drift, y_pred_drift),
    "f1_macro": f1_score(y_drift, y_pred_drift, average="macro")
}

if hasattr(model, "predict_proba"):
    y_prob_drift = model.predict_proba(X_drift)
    drift_metrics["roc_auc_ovo"] = roc_auc_score(
        y_drift, y_prob_drift, multi_class="ovo"
    )

drift_metrics

Comparación de métricas

In [ ]:
metrics_comparison = pd.DataFrame({
    "baseline": baseline_metrics,
    "with_drift": drift_metrics
})
metrics_comparison

Gráficas de distribuciones (como la imagen roja/azul)

In [ ]:
# Elegimos algunas variables numéricas para ilustrar
cols_to_plot = drift_num["variable"].head(3).tolist() if not drift_num.empty else num_cols[:3]

for col in cols_to_plot:
    plt.figure()
    plt.hist(X[col], bins=40, alpha=0.5, label="original")
    plt.hist(X_drift[col], bins=40, alpha=0.5, label="drift")
    plt.title(f"Distribución - {col}")
    plt.legend()
    plt.show()

Curvas ROC y PR (original vs drift)

In [ ]:
if hasattr(model, "predict_proba"):
    # ROC original
    RocCurveDisplay.from_estimator(model, X, y)
    plt.title("ROC - Datos originales")
    plt.show()

    # ROC drift
    RocCurveDisplay.from_estimator(model, X_drift, y_drift)
    plt.title("ROC - Datos con drift")
    plt.show()

    # PR original
    PrecisionRecallDisplay.from_estimator(model, X, y)
    plt.title("PR - Datos originales")
    plt.show()

    # PR drift
    PrecisionRecallDisplay.from_estimator(model, X_drift, y_drift)
    plt.title("PR - Datos con drift")
    plt.show()